# Day 9

[Day 9 description](https://adventofcode.com/2018/day/9)

Oh boy, this was hard to optimize. And I think I did a good job overall anyway. Instead of adding one marble at a time, I was able to implement a function to add 23 marbles at a time, which is easier. It's even easy to compute the id of the player at every 23th step. So, most of the time is spent in creating the whole list of numbers.

Most probably, I should have implemented a circular queue. In the mean time, I always rotate the array in order to have the last marble in last position, which keeps the logic cleaner.

In [1]:
from collections import defaultdict
import numpy as np

In [2]:
with open('AOC2018_09_input.txt') as f:
    raw = f.read().strip()
raw

'447 players; last marble is worth 71510 points'

In [3]:
n_of_players = 447
last_marble = 71510

In [4]:
def player_number(marble_number, number_of_players):
    return marble_number % number_of_players

def best_score(scores, n):
    # here's a clever bit: since I have n players, every n-th score belongs to the same player
    # so scores[i::n] is the list of scores for player i
    return max(sum(scores[i::n]) for i in range(n))

def rotate(xs, p):
    return xs[p:] + xs[:p]

def rotate_and_remove(xs):
    xs, m = xs[-7:] + xs[:-8], xs[-8]
    return rotate(xs, 1), m

def flatten(pairs):
    return [x for pair in pairs for x in pair]

def add_23_marbles(xs, last_marble):
    new_last_marble = last_marble + 23
    if len(xs) < 23:
        for i in range(last_marble+1, last_marble+23):
            xs = rotate(xs, 1)
            xs.append(i)
    else:
        # here's a clever bit: since I know how to insert the next 23 marbles,
        # I can create build this in one step
        new_marbles = range(last_marble + 1, new_last_marble)
        xs = xs[22:] + flatten(zip(xs[:22], new_marbles))
    xs, marble = rotate_and_remove(xs)
    points = marble + new_last_marble
    return xs, points

In [5]:
xs_ = [0]
scores = []
for i in range(last_marble//23):
    xs_, m = add_23_marbles(xs_, 23*i)
    scores.append(m)
best_score(scores, n_of_players)

398242

In [6]:
xs_ = [0]
scores = []
for i in range(100*last_marble//23):
    if i % 10000 == 0:
        print(i)
    xs_, m = add_23_marbles(xs_, 23*i)
    scores.append(m)
best_score(scores, n_of_players)

0


KeyboardInterrupt: 

In [7]:
# ...a couple of hours later: 3273842452

A slightly faster approach is with numpy. The logic is the same, still matter of hours

In [8]:
import numpy as np

In [9]:
def rotate_np(xs, p):
    return np.roll(xs, -p)

def rotate_and_remove_np(xs):
    res = np.zeros(xs.size - 1)
    res[:6] = xs[-6:]
    res[6:-1] = xs[:-8]
    res[-1] = xs[-7]
    return res, xs[-8]

def rotate_and_remove_np_1(xs):
    res = np.zeros(xs.size - 1)
    res[:7] = xs[-7:]
    res[7:] = xs[:-8]
    return rotate_np(res, 1), xs[-8]

In [10]:
def add_23_marbles_np(xs, last_marble):
    new_last_marble = last_marble + 23
    if xs.size < 40:
        for i in np.arange(last_marble+1, last_marble+23):
            xs = rotate_np(xs, 1)
            xs_new = np.zeros(xs.size+1)
            xs_new[:xs.size] = xs
            xs_new[-1] = i
            xs = xs_new
    else:
        new_marbles = np.arange(last_marble + 1, new_last_marble)
        res = np.zeros(xs.size + 22)
        res[:-44] = xs[22:]
        res[-44::2] = xs[:22]
        res[-43::2] = new_marbles
        xs = res
    xs, marble = rotate_and_remove_np(xs)
    points = marble + new_last_marble
    return xs, points

In [11]:
last_marble = 7151000
xs_ = np.array([0])
scores = []
for i in range(last_marble//23):
    if i % 10000 == 0:
        print(i)
    xs_, m = add_23_marbles_np(xs_, 23*i)
    scores.append(m)
best_score(scores, n_of_players)

0
10000


KeyboardInterrupt: 